# Install requirements and set up Bedrock client

In [ ]:
%pip install -r requirements.txt

In [ ]:
import boto3
import base64
import json
import time
import random
import sagemaker

from helpers.bedrock_helpers import generate_text, generate_images, get_random_seed, get_task_status
from helpers.display_helpers import display_story_table, display_prompt_table, display_storyboard, display_hyperlink
from helpers.prompt_helpers import system_prompts, get_character_descriptions, get_style_prompt

bedrock_client = boto3.client("bedrock-runtime")
bedrock_client_us_east = boto3.client("bedrock-runtime", region_name="us-east-1")

session = sagemaker.Session()
default_bucket = session.default_bucket()
role = sagemaker.get_execution_role()

### Make sure the role below has the following permissions:
- bedrock:invoke_model
- bedrock:GetAsyncInvoke

### You can use the following policy:

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel",
                "bedrock:GetAsyncInvoke"
            ],
            "Resource": "*"
        }
    ]
}
```

In [ ]:
display_hyperlink(role, f"https://us-east-1.console.aws.amazon.com/iam/home?region=us-east-1#/roles/details/{role.split("/")[-1]}?section=permissions")

# Create the story and characters using generative AI

In [ ]:
scene_count = 3
genre = "Family"
idea = "a small robot is lost in a library and needs to read some books to find its way out"
characters = []

In [ ]:
user_prompt = f"""
        Human: You are an award wining director, create {scene_count} scenes from the <StoryIdea> for storyboarding. The genre is {genre}.
            <StoryIdea>
            {idea}
            </StoryIdea>
        Use the specified characters in the story.  You can add more if needed.
            <Characters>
            {json.dumps(characters)}
            </Characters>
"""

In [ ]:
story = generate_text(bedrock_client, user_prompt, system_prompts["story"])
display_story_table(story)

# For each scene, create a prompt to be used with Nova Canvas to generate storyboard panels

In [ ]:
style = "3D"
throttle_pause_time = 0

image_prompts = {}
for scene in story.get("scenes"):
    print("Processing scene {}/{} ...".format(scene["scene_id"]+1, story.get("scene_count")))
    base_image_prompt = f"""Generate a storyboard image for the following scene:
            Scene decription:
            {scene["description"]}
            Scene imagary:
            {scene["imagery"]}
            Character descriptions:
            {get_character_descriptions(scene)}
            """
    conditioned_image_prompt = generate_text(bedrock_client, base_image_prompt, system_prompts["image"])
    time.sleep(throttle_pause_time)
    styled_image_prompt = generate_text(bedrock_client, f"Image generation prompt: {conditioned_image_prompt}", get_style_prompt(style))
    image_prompts[scene["scene_id"]] = styled_image_prompt["prompt"]
    time.sleep(throttle_pause_time)

display_prompt_table(story_data=story, image_prompts=image_prompts)

# Use the generated prompts to create multiple variations of a storyboard panel for each scene

In [ ]:
images_per_scene = 3
negative_prompt = "ugly, yellow, red, green, blue, pink, orange, brown, old, dirty, colorful"
seed = get_random_seed()

storyboard_images = {}
for scene in story.get("scenes"):
    scene_id = scene["scene_id"]
    print("Processing scene {}/{} ...".format(scene_id+1, story.get("scene_count")))
    prompt = image_prompts[scene_id]
    images = generate_images(bedrock_client_us_east, prompt, negative_prompt, seed=seed, image_count=images_per_scene)
    storyboard_images[scene_id] = images
display_storyboard(storyboard_images, story)

# Create a video from the first image of the first scene

In [ ]:
video_prompt = generate_text(bedrock_client, image_prompts[0], system_prompts["video"]).get("prompt")
model_input = {
    "taskType": "TEXT_VIDEO",
    "textToVideoParams": {
        "text": video_prompt,
        "images": [{ "format": "png", "source": { "bytes": storyboard_images[0][0]} }]
    },
    "videoGenerationConfig": {
        "durationSeconds": 6,
        "fps": 24,
        "dimension": "1280x720",
        "seed": seed
    }
}

invocation = bedrock_client_us_east.start_async_invoke(
    modelId="amazon.nova-reel-v1:0",
    modelInput=model_input,
    outputDataConfig={"s3OutputDataConfig": {"s3Uri": f"s3://{default_bucket}"}}
)

invocation_arn = invocation["invocationArn"]
s3_prefix = invocation_arn.split('/')[-1]
s3_location = f"s3://{default_bucket}/{s3_prefix}"
print(f"\nS3 URI: {s3_location}")


In [ ]:
status = get_task_status(bedrock_client_us_east, invocation_arn)
print(f"Status: {status}")